In [ ]:
# WE use yahoo_fin and yfinance as modules to read stock information.
import pandas as pd
import numpy as np
import yahoo_fin.stock_info as si
import yfinance as yf

In [ ]:
# FIRST READ ALL STOCK TICKERS (OR MOST OF THEM, THE MOST RELEVANT)
# we use the yahoo_fin.stock_info module in order to receive lists of ticker symbols.
# we get approx. 8900 symbols (can change any day as it is updated constantly..)
ticker1 = si.tickers_other()
ticker2 = si.tickers_sp500()
ticker3 = si.tickers_dow()
ticker4 = si.tickers_nasdaq()
infirst = set(ticker1)
insecond = set(ticker2)
result2 = ticker1 + list(insecond - infirst)
infirst = set(result2)
insecond = set(ticker3)
result3 = result2 + list(insecond - infirst)
infirst = set(result3)
insecond = set(ticker4)
all_tickers = result3 + list(insecond - infirst)
all_tickers  # Now we have a list of stock symbols, with which we can use 
# the yfinance module to read info about them. (approx. 8900 symbols)

In [ ]:
# READ DATA FROM yfinance module. In both cases of training and test we tried to find
# a period of time in which the economy is more or less stable, and the stock market
# is more or less healthy. This way we can ensure that we try to learn the stock BEHAVIOR
# without the effects of global issues.
# For LEARNING, we used the period of 01.01.2011 to 31.12.2019
# For TEST, we used period of 01.04.2003 to 31.05.2007. 
data_from_yfinance = yf.download(all_tickers, start='2011-01-01', end='2019-12-31')

In [ ]:
# save read data, used internaly for us. Used different name for test
data_from_yfinance.to_pickle('raw_data_save.pkl')

In [ ]:
# read saved data, used inernaly for us. Used different name for test
data_from_yfinance = pd.read_pickle('raw_data_save.pkl')

In [ ]:
# Begin cleaning the data, extracting the information that we need.  
# extract only close and volume. From close we will derive 4 more indicators, so we will
# have 6 indicators in total for each stock.

close = data_from_yfinance['Close']
volume = data_from_yfinance['Volume']

In [ ]:
# clean the data. remove anything that isnt with full info of the given period.
# each year has some days that the stock market is completely closed. In those days we have NaN values
# so we remove them.
close_clean = close.dropna(axis=0, how='all')  
# Any stock that doesnt have full information of the given period will also be dropped.
close_clean = close_clean.dropna(axis=1, how='any')  
# Remove the date indication. Now each column is a stock, and rows are consecutive days
close_clean = close_clean.reset_index(drop=True)
# save the close price data.
close_clean.to_csv('C:\\Users\\Daniel\\PycharmProjects\\StockData\\close_clean.csv', index=False)

In [ ]:
# same here, for volume. 
# we note that the columns and rows of close and volume correspond completely. 
# where there are values in close there are also values in volume, and vice versa.
volume_clean = volume.dropna(axis=0, how='all')
volume_clean = volume_clean.dropna(axis=1, how='any')
volume_clean = volume_clean.reset_index(drop=True)
volume_clean.to_csv('C:\\Users\\Daniel\\PycharmProjects\\StockData\\volume_test.csv', index=False)

In [ ]:
# build the indicators. We decided to use the MA50 - 50 day moving average, MA200 - 200 day moving average,
# EMA200 - 200 day exponential moving average, RSI - relative strength index. In total, with the close
# price indicator and volume indicator we will have 6 valid indicators for all the stocks, for the 
# given period of time.
# Here we calculate MA50, and save it for use internaly.
ma50 = close_clean.rolling(50).mean()
ma50.to_csv('C:\\Users\\Daniel\\PycharmProjects\\StockData\\ma50_test.csv', index=False)

In [ ]:
# Here we calculate MA200
ma200 = close_clean.rolling(200).mean()
ma200.to_csv('C:\\Users\\Daniel\\PycharmProjects\\StockData\\ma200_test.csv', index=False)

In [ ]:
# Here we calculate the EMA200
ema200 = close_clean.ewm(span=200).mean()
ema200.to_csv('C:\\Users\\Daniel\\PycharmProjects\\StockData\\ema200_test.csv', index=False)

In [ ]:
# Here we calculate the RSI
delta = close_clean.diff()
up, down = delta.copy(), delta.copy()
up[up < 0] = 0
down[down > 0] = 0
roll_up = up.rolling(14).mean()
roll_down = down.abs().rolling(14).mean()
RSI = 100.0 - (100.0/(1.0 + (roll_up/roll_down)))
RSI.to_csv('C:\\Users\\Daniel\\PycharmProjects\\StockData\\RSI_test.csv', index=False)


In [ ]:
# read currently available indicator, for internal use for us.
close_clean = pd.read_csv('close_clean.csv')
ma50 = pd.read_csv('ma50.csv')
RSI = pd.read_csv('RSI.csv')
ema200 = pd.read_csv('ema200.csv')
ma200 = pd.read_csv('ma200.csv')
volume = pd.read_csv('volume.csv')

In [ ]:
# Start the trimming process to create the actual database
# because we want to use the 200 day moving average, the first 200 days of all the data is irrelevant 
# because it wont have the 200MA indicator
# find the range of indexes relevant
first_index = ma200.first_valid_index()
last_index = ma200.last_valid_index()

In [ ]:
# Remove 200 first days of all data so that it corresponds to no data of MA200

close_trimmed = close_clean.iloc[first_index:last_index, :].reset_index(drop=True)
ma50_trimmed = ma50.iloc[first_index:last_index, :].reset_index(drop=True)
ma200_trimmed = ma200.iloc[first_index:last_index, :].reset_index(drop=True)
ema200_trimmed = ema200.iloc[first_index:last_index, :].reset_index(drop=True)
RSI_trimmed = RSI.iloc[first_index:last_index, :].reset_index(drop=True)
volume_trimmed = volume_clean.iloc[first_index:last_index, :].reset_index(drop=True)

In [ ]:
# Divide the trimmed data into 9 approximate years. generate random offsets for each stock
# We want to use a 'more-or-less' year's worth of information about the stock in order to learn about it
# and derive whether the next year will be at least 10% profitable from the last day of the 
# current year. 
# We had approx. 9 years of information, and removed the first 200 days. After some calculations, we decided
# that a year will be exactly 229 days (which is close enough to an actual year - approx. 256 days of trading,
# excluding saturday and sunday, and some non-trading days).
# These calculations were performed early on in the work so we decided to stick with them.
# Now, if we had divided the information as is into 229 day periods, there could be a lot of 
# correlation between stock of the same period, if for example those 229 days were strong days in the 
# stock market. So, in order to remove the correlation, we decided to offset each stock information
# by some random value between 0 to 229. Here, we generate the offset value for each stock
year_day_size = 229  # should be 229, or total/9
start_indexes_for_stocks = np.random.randint(low=0, high=year_day_size, size=len(close.columns))
start_indexes_for_stocks

In [ ]:
# We shift each stock to stock[offset:-(year-offset), so that we are left with one less year of information
# for each stock, and the stock are uncorrelated with regards to date.
# after this, each stock contains 8 clean years(229) of info
offseted_close = pd.DataFrame(columns=close_trimmed.columns)
offseted_ma50 = pd.DataFrame(columns=close_trimmed.columns)
offseted_ma200 = pd.DataFrame(columns=close_trimmed.columns)
offseted_ema200 = pd.DataFrame(columns=close_trimmed.columns)
offseted_RSI = pd.DataFrame(columns=close_trimmed.columns)
offseted_volume = pd.DataFrame(columns=close_trimmed.columns)
for i in np.arange(len(offseted_close.columns)):
    offseted_close.iloc[:, i] = np.array(close_trimmed.iloc[start_indexes_for_stocks[i]:-(year_day_size-start_indexes_for_stocks[i]), i])
    offseted_ma50.iloc[:, i] = np.array(ma50_trimmed.iloc[start_indexes_for_stocks[i]:-(year_day_size-start_indexes_for_stocks[i]), i])
    offseted_ma200.iloc[:, i] = np.array(ma200_trimmed.iloc[start_indexes_for_stocks[i]:-(year_day_size-start_indexes_for_stocks[i]), i])
    offseted_ema200.iloc[:, i] = np.array(ema200_trimmed.iloc[start_indexes_for_stocks[i]:-(year_day_size-start_indexes_for_stocks[i]), i])
    offseted_RSI.iloc[:, i] = np.array(RSI_trimmed.iloc[start_indexes_for_stocks[i]:-(year_day_size-start_indexes_for_stocks[i]), i])
    offseted_volume.iloc[:, i] = np.array(volume_trimmed.iloc[start_indexes_for_stocks[i]:-(year_day_size-start_indexes_for_stocks[i]), i])
# we have slightly more than exactly 8 years of clean data, so we trim it some more below

In [ ]:
# change a bit so that info is exactly 8 years, when year = 229 days (data points)
# NOTE: if we were addressing testing data, then we would have 2 years of clean data.
offseted_volume = offseted_volume.iloc[:1832]  # 458 for test, 1832 for actual data = 229*8
offseted_RSI = offseted_RSI.iloc[:1832]  # same for rest
offseted_ma200 = offseted_ma200.iloc[:1832]
offseted_ma50 = offseted_ma50.iloc[:1832]
offseted_ema200 = offseted_ema200.iloc[:1832]
offseted_close = offseted_close.iloc[:1832]

In [ ]:
# save the randomly offseted stock and their indicators, For internal use.
offseted_close.to_csv('offseted_close.csv', index=False)
offseted_ma50.to_csv('offseted_ma50.csv', index=False)
offseted_ema200.to_csv('offseted_ema200.csv', index=False)
offseted_ma200.to_csv('offseted_ma200.csv', index=False)
offseted_RSI.to_csv('offseted_RSI.csv', index=False)
offseted_volume.to_csv('offseted_volume.csv', index=False)

In [ ]:
# read the offseted data, For internal use.
offseted_close = pd.read_csv('offseted_close.csv')
offseted_ma50 = pd.read_csv('offseted_ma50.csv')
offseted_ma200 = pd.read_csv('offseted_ma200.csv')
offseted_ema200 = pd.read_csv('offseted_ema200.csv')
offseted_RSI = pd.read_csv('offseted_RSI.csv')
offseted_volume = pd.read_csv('offseted_volume.csv')

In [ ]:
# now that the data is exactly 8 years of 229 days split each indicator into 8 years
# We split each indicator so that we have 8 lists, of DataFrames of 1 year of each indicator 
# for each stock
close_dfs = [offseted_close.iloc[i:i+year_day_size] for i in np.arange(0, offseted_close.shape[0], year_day_size)]
ma50_dfs = [offseted_ma50.iloc[i:i+year_day_size] for i in np.arange(0, offseted_close.shape[0], year_day_size)]
ema200_dfs = [offseted_ema200.iloc[i:i+year_day_size] for i in np.arange(0, offseted_close.shape[0], year_day_size)]
ma200_dfs = [offseted_ma200.iloc[i:i+year_day_size] for i in np.arange(0, offseted_close.shape[0], year_day_size)]
RSI_dfs = [offseted_RSI.iloc[i:i+year_day_size] for i in np.arange(0, offseted_close.shape[0], year_day_size)]
volume_dfs = [offseted_volume.iloc[i:i+year_day_size] for i in np.arange(0, offseted_close.shape[0], year_day_size)]

In [ ]:
# Using the above lists, we build the final Data set for the actual learning the testing.
# Now the target is did we achieve 10% profit sometime in the next year, with regards to the last
# day of the current year. Note that the last year cannot be checked as we dont have the 
# information of the stock prices for the next year. So, after this process, we are left with 7 
# years of learnable, usable, information (1 year for test)
# The data set looks as follows: it has a ticked symbol column(each ticker is repeated 7 times),
# a close column (raw close price), ma50, ma200, ema200, RSI, volume, and target.
# target is set to 1 if we achieved 10% profit as stated above, or 0 otherwise.
# Note, that in the columns of the indicators each cell contains a full list of consecutive information, of size 229.
complete_df = pd.DataFrame(columns=('ticker', 'close', 'ma50', 'ma200', 'ema200', 'RSI', 'volume', 'target'))
years = len(close_dfs) - 1 # this is 7 for data, 1 for test.
for i in np.arange(len(offseted_close.columns), dtype=int):  # for each stock
    for j in np.arange(years):  # for each year of info except last, because we dont know target
        if (close_dfs[j+1].iloc[:, i] / close_dfs[j].iloc[-1, i] > 1.1).any():
            target = 1
        else: 
            target= 0
        complete_df.at[i*years + j, 'ticker'] = offseted_close.columns[i] 
        complete_df.at[i*years + j, 'close'] = list(close_dfs[j].iloc[:,i])
        complete_df.at[i*years + j, 'ma50'] = list(ma50_dfs[j].iloc[:,i])
        complete_df.at[i*years + j, 'ma200'] = list(ma200_dfs[j].iloc[:,i])
        complete_df.at[i*years + j, 'ema200'] = list(ema200_dfs[j].iloc[:,i])
        complete_df.at[i*years + j, 'RSI'] = list(RSI_dfs[j].iloc[:,i])
        complete_df.at[i*years + j, 'volume'] = list(volume_dfs[j].iloc[:,i])
        complete_df.at[i*years + j, 'target'] = target

In [ ]:
# save the database, for internal use. We use pickle as it saves without any losses, or problems,
# that we otherwise had with csv.
# test_set for test, data_set for data
complete_df.to_pickle('data_set.pkl')
# This concludes the process of raw building of the indicators, and stock information.